# IV Estimation of Quantile Treatment Effects
The goal of this notebook is to reproduce the results from III. 7.2 in Mostly Harmless Econometrics (MHE) by Angrist&Pischke.

## 0) Import & Convenience Functions

In [8]:
import os
import pandas
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit
from linearmodels import IV2SLS
from sklearn.linear_model import QuantileRegressor, LogisticRegression
from sklearn.model_selection import cross_validate
import numpy
from sklearn.ensemble import RandomForestClassifier
from IPython.display import clear_output

@property
def params(self):
    return pandas.Series(data = self.coef_, index=self.feature_names_in_)
QuantileRegressor.params = params

class ProbitWrapper:
    """Provides sklearn API for statsmodels Probit"""
        
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.model = Probit(y, X)
        self.res = self.model.fit(max_iter=200, method= "newton")
    
    def predict_proba(self, X):
        p1 = self.res.predict(X)
        p0 = 1-p1
        return pandas.concat([p0, p1], axis=1).values
    
    def predict(self,X):
        return self.res.predict(X)
    
    def get_params(self, deep=False):
        return {}
    @property
    def classes_(self):
        return numpy.array([0,1])
    
RF_KWARGS_DEF = {
    "n_estimators": 500, 
    "criterion":'gini',
    "max_depth":None,
    "min_samples_split":2,
    "min_samples_leaf":1,
    "min_weight_fraction_leaf":0.0,
    "max_features":'auto',
    "max_leaf_nodes":None,
    "min_impurity_decrease":0.0,
    "bootstrap":True,
    "oob_score":False,
    "n_jobs":None,
    "random_state":None,
    "verbose":0,
    "warm_start":False,
    "class_weight":None,
}

RF_KWARGS_MAX = {k:v for k,v in RF_KWARGS_DEF.items()}
RF_KWARGS_MAX["max_depth"] = 10000
RF_KWARGS_MAX["bootstrap"] = False

RF_KWARGS_1 = {k:v for k,v in RF_KWARGS_DEF.items()}
RF_KWARGS_1["max_depth"] = 4
RF_KWARGS_1["min_samples_split"] = 5
RF_KWARGS_1["min_samples_leaf"] = 5
RF_KWARGS_1["oob_score"] = True
RF_KWARGS_1["max_samples"] = .7

RF_KWARGS_2 = {k:v for k,v in RF_KWARGS_DEF.items()}
RF_KWARGS_2["max_depth"] = 8
RF_KWARGS_2["min_samples_split"] = 5
RF_KWARGS_2["min_samples_leaf"] = 5
RF_KWARGS_2["oob_score"] = True
RF_KWARGS_2["max_samples"] = .7

RF_KWARGS_3 = {k:v for k,v in RF_KWARGS_DEF.items()}
RF_KWARGS_3["max_depth"] = 16
RF_KWARGS_3["min_samples_split"] = 5
RF_KWARGS_3["min_samples_leaf"] = 5
RF_KWARGS_3["max_samples"] = .7
RF_KWARGS_3["oob_score"] = True

RF_KWARGS_4 = {k:v for k,v in RF_KWARGS_DEF.items()}
RF_KWARGS_4["max_depth"] = 16
RF_KWARGS_4["min_samples_split"] = 3
RF_KWARGS_4["min_samples_leaf"] = 3
RF_KWARGS_4["max_samples"] = .7
RF_KWARGS_4["oob_score"] = True

We use two convenience functions to get results (parameters, standard errors) from a model and produce a tabular overview. The typical output of *get_result* looks like the following:

```python
{'type': 'OLS',
 'param_type': 'coef',
 'hsorged': 4015.4,
 'black': -2354.2,
 'hispanic': 250.9,
 'married': 6545.7,
 'wkless13': -6581.7
 'class_tr': 5945.9
 'ojt_jsa': 7205.0
 'age2225': 5736.8
 'age2629': 4603.9,
 'age3035': 2125.4,
 'age3644': -1644.5,
 'age4554': 454.5,
 'f2sms': 2101.6.
 'constant': 9810.6,
 'd': 3753.6}ˋ
```

The function *get_final_table* takes a list of such records and produces a table where for each analysis run (e.g. OLS) the coefficient and standard error of each variable is reported.

In [2]:
def get_result(Model, meta, model_kwargs, fit_kwargs, params_attr="params", bse_attr= "bse"):
    fit_kwargs = fit_kwargs if fit_kwargs is not None else {}
    model = Model(**model_kwargs)
    res = model.fit(**fit_kwargs)
    params = getattr(res, params_attr)
    params_res = [{**meta, "param_type": "coef", **{f"{k}": v for k,v in params.to_dict().items()}}]
    if bse_attr is not None:
        dev = getattr(res, bse_attr)
        dev_res = [{**meta, "param_type": "se", **{f"{k}": v for k,v in dev.to_dict().items()}}]
    else:
        dev_res = []
    return params_res+dev_res

def get_final_table(df, add_kappa_cv=False):

    df_results = df.melt(id_vars=["type", "param_type"], )
    df_results = df_results.pivot_table(index=["variable", "param_type"], columns=["type"], aggfunc="first")    

    display_columns=[
        ("d", "coef"), ("d", "se"),
        ("hsorged", "coef"), ("hsorged", "se"),
        ("black", "coef"), ("black", "se"),
        ("hispanic", "coef"), ("hispanic", "se"),
        ("married", "coef"), ("married", "se"),
        ("wkless13", "coef"), ("wkless13", "se"),
        ("constant", "coef"), ("constant", "se"),
    ]
    if add_kappa_cv:
        display_columns+=[
                    ("kappa_cv0", "coef"), ("kappa_cv0", "se"),
                    ("kappa_cv1", "coef"), ("kappa_cv1", "se")
                    ]
    
    return df_results.round(3).T[display_columns].T

        

A crucial step in the IV estimation is the calculation of $E[\kappa_i|Y_i, D_i, X_i]$ (see MHE (7.2.5), p.287) which we implement in the function *get_kappa*. The main work is actually the computation of $E[Z_i|Y_i, D_i, X_i]$ which requires a base model. For such, currently implemented options are Probit, Logit, and RandomForest.

To allow for hyper parameter tuning and robustness checks, another output are cross validation log losses (separately for $D=0$ and $D=1$) for the model fit. 
Moreover, it is possible to pass additional model paramaters in kwargs['model_kwargs'] as well as a degree in kwargs['deg'] to allow for polynomial features up to the specified degree.

Remark: In the original paper (Abadie, Angrist, Imbens, 2002) a polynomial probit model is used (plus additional feature selection).

In [3]:
def fit_model(model_name, model_kwargs, X, y):
    modelname2model = {"probit": ProbitWrapper, "rf": RandomForestClassifier, "logit": LogisticRegression}
    Model = modelname2model[model_name]
    model = Model(**model_kwargs)
    model.fit(y=y, X=X)
    pred = model.predict_proba(X)[:,1]
    pred = pandas.Series(pred, X.index)
    cv = cross_validate(model, X, y, cv=5, scoring="neg_log_loss")["test_score"]
    return pred, cv

def get_kappa(xs,ys,zs,ds, model_name, kwargs, return_pred=False, return_raw=False):
    
    
    assert all(xs.index==ys.index)
    assert all(ys.index==zs.index)
    assert all(zs.index==ds.index)
    
    p_z_given_y_x_d0 = 0*ds + 1000*0
    p_z_given_y_x_d1 = 0*ds + 1000*0
    p_z_given_x = 0*ds + 1000*0
    all_pred =[]
    all_cv = []
    deg = kwargs["deg"]
    model_kwargs = kwargs["model_kwargs"]
    for d in [0,1]:
        m_d = ds==d
        y, X = zs[m_d], pandas.concat([xs[m_d]] + [ys[m_d]**k for k in range(1, deg+1)], axis=1)
        
        pred, cv = fit_model(model_name, model_kwargs, X, y)
        all_pred.append(pred[m_d])

        if d==0:
            p_z_given_y_x_d0.loc[m_d] = pred[m_d]
            cv0 = numpy.mean(-cv)
        elif d==1:
            p_z_given_y_x_d1.loc[m_d] = pred[m_d]
            cv1 = numpy.mean(-cv)
        else:
            raise AssertionError("")
    
    y, X = zs, xs
    X = X[list(X.columns)[0:1]]*0+1 # comment in paper
    pred, cv = fit_model(model_name, model_kwargs, X, y)
    all_pred.append(pred)
    
    p_z_given_x = pred
                
    e_kappa = ds*0+1
    e_kappa -= ds*(1-p_z_given_y_x_d1)*1/(1e-3+(1-p_z_given_x))
    e_kappa -= (1-ds)*(p_z_given_y_x_d0)*1/(1e-3+p_z_given_x)
    if not return_raw:
        e_kappa = e_kappa.clip(0,1)
        e_kappa = e_kappa.fillna(e_kappa.median())
        e_kappa = e_kappa.fillna(1)
    if return_pred:
        return e_kappa, all_pred
    return e_kappa, cv0, cv1
        

## 1) Loading Dataset

In [4]:
import requests

url = "https://economics.mit.edu/sites/default/files/publications/jtpa.raw"
u = requests.get(url)
lines = u.content.decode("utf-8").split("\r\n")

lines = [l.replace("\n", "") for l in lines]
data = [[x for x in l.split(" ") if x!=""] for l in lines if l!=""]
df = pandas.DataFrame(data)
df = df.reset_index(drop=True)

columns= [
    (1, "index"),
    (2, "y"), 
    (3, "z"), 
    (4, "d"),
    (5, "sex"),
    (6, "hsorged"),
    (7, "black"),
    (8, "hispanic"),
    (9, "married" ),
    (10, "wkless13"),
    (11, "afdc"),
    (17, "class_tr"),
    (18, "ojt_jsa"),
    (12, "age2225"),
    (13, "age2629"),
    (14, "age3035"),
    (15, "age3644"),
    (16, "age4554"),
    (19, "f2sms")]
    
_, columns = zip(*columns)

df.columns = columns

y_col = "y"
d_col = "d"
z_col = "z"

x_cols_m = [
    "hsorged", 
    "black", 
    "hispanic", 
    "married", 
    "wkless13",
    "class_tr", 
    "ojt_jsa",
    "age2225", 
    "age2629", 
    "age3035", 
    "age3644", 
    "age4554",
    "f2sms",
    "constant",
]

x_cols_f = [
    "hsorged", 
    "black", 
    "hispanic", 
    "married", 
    "wkless13",
    "afdc",
    "class_tr", 
    "ojt_jsa",
    "age2225", 
    "age2629", 
    "age3035", 
    "age3644", 
    "age4554",
    "f2sms",
    "constant",
]

df["constant"] = 1
float_cols = ["y", "d", "z", "sex"] + list(set(x_cols_f+x_cols_m))
df[float_cols] = df[float_cols].astype("float")

## 2) OLS and Quantile Regression
Here we reprodue part A of table 7.2.1 in MHE. For example the training effect of 3754 (536) in this table is undistinguishable from the analogous coefficient of d (3754 (535)) computed below.


In [5]:
all_results = []

m = df["sex"]==1
df_ = df[m]
y = df_[y_col]
X = df_[x_cols_m + [d_col]]


this_result = get_result(sm.OLS, {"type": "OLS"}, model_kwargs={"endog":y, "exog": X}, fit_kwargs = {"method":"pinv"},
           params_attr="params", bse_attr= "bse")
all_results += this_result


model = sm.QuantReg(y, X)
qs = [.15, .25, .5, .75, .85]

for q in qs:
    this_result = get_result(sm.QuantReg, {"type": f"QR{q}"}, model_kwargs={"endog":y, "exog": X}, fit_kwargs = {"q": q, "p_tol": 1e-6, "kernel":'epa', "bandwidth":'hsheather', "max_iter":1000,},
           params_attr="params", bse_attr= "bse")
    all_results+=this_result
    
    
    #this_result = get_result(QuantileRegressor, {"type": f"SKQR{q}"},
    #                            model_kwargs={"quantile": q, "alpha":0, "solver": "highs-ds", "fit_intercept":False},
    #                            fit_kwargs={"X": X, "y": y},
    #                            params_attr = "params", bse_attr=None)
    # --> same result
    
    all_results+=this_result
    
df_results_simple_regression = pandas.DataFrame(all_results)
get_final_table(df_results_simple_regression)

/data/user/0/ru.iiec.pydroid3/files/aarch64-linux-android/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


value                                           \
type                      OLS    QR0.15    QR0.25     QR0.5     QR0.75   
variable param_type                                                      
d        coef        3753.648  1187.440  2509.721  4411.000   4678.039   
         se           534.540   224.245   308.466   593.711    905.609   
hsorged  coef        4015.431   339.344  1279.543  3650.875   6045.346   
         se           594.846   249.456   342.160   660.692   1004.679   
black    coef       -2354.207  -134.486  -500.460 -2079.000  -3575.771   
         se           640.492   269.185   370.032   711.391   1073.881   
hispanic coef         250.929    90.626   278.181   919.875   -876.811   
         se           907.362   375.560   521.350  1007.803   1536.099   
married  coef        6545.679   586.932  1963.804  7127.376  10072.654   
         se           583.646   241.819   334.725   648.253    992.306   
wkless13 coef       -6581.672 -1089.716 -3097.101 -7615.125  -9833.961   
         se           572.810   242.295   331.327   636.217    973.423   
constant coef        9810.577  -216.904   364.743  6119.000  14873.961   
         se          1646.256   665.632   936.471  1828.489   2851.205   

                                
type                    QR0.85  
variable param_type             
d        coef         4806.091  
         se           1010.030  
hsorged  coef         6223.783  
         se           1123.818  
black    coef        -3609.325  
         se           1211.847  
hispanic coef          -85.154  
         se           1720.263  
married  coef        11061.983  
         se           1119.810  
wkless13 coef        -9951.154  
         se           1096.711  
constant coef        21526.651  
         se           3216.018

## 3) IV Estimation
Now we look at the IV estimation. We use IV2SLS from linearmodels for the 2SLS estimates and QuantileRegressor from sklearn with weights given by the computed $\kappa$. Standard errors are estimated based on bootstrapping.

To analyze the sensitivity of the results against the fit of $\kappa$ we run the analysis for several choices of models and hyper-parameters.


In [ ]:
from scipy.stats import boxcox

all_results_bootstrap_2slsqte = []
for i in range(0, 25, 1):    
    for d in [1]:
        m = df["sex"]==d
        df_ = df[m].sample(n=m.sum(), replace=True, random_state=numpy.random.RandomState()).reset_index(drop=True)

        model_kwargs = {"dependent":df_[y_col], 
                       "exog": df_[x_cols_m], 
                       "endog": df_[d_col], 
                       "instruments": df_[z_col]}
        
        this_result = get_result(IV2SLS,{"type": f"2SLS(sex={d})"},
                                model_kwargs=model_kwargs, fit_kwargs={},
                                 params_attr = "params",bse_attr=None)
        all_results_bootstrap_2slsqte+=this_result

        ys = df_[y_col]
        
        xs = df_[x_cols_m]
        zs = df_[z_col]
        ds = df_[d_col]
        for model_name, model_kwargs,log_ykappa, model_str in [
                                        #("logit", {"deg": 1, "model_kwargs": {}},True, "logit_1"), 
                                        #("logit", {"deg": 3, "model_kwargs": {}},True, "logit_3"), 
                                        ("logit", {"deg": 7, "model_kwargs": {}},False, "logit_7"), 
                                        #("logit", {"deg": 1, "model_kwargs": {}},False, "logit_1"), 
                                        #("logit", {"deg": 3, "model_kwargs": {}},False, "logit_3"), 
                                        #("logit", {"deg": 5, "model_kwargs": {}},False, "logit_5"), 
                                        #("probit", {"deg": 1, "model_kwargs": {}},False, "probit_1"), 
                                        #("probit", {"deg": 3, "model_kwargs": {}},False, "probit_3"), 
                                        #("probit", {"deg": 7, "model_kwargs": {}},True, "probit_5"),
                                        #("probit", {"deg": 1, "model_kwargs": {}},True, "probit_1"), 
                                        #("probit", {"deg": 3, "model_kwargs": {}},True, "probit_3"), 
                                        #("probit", {"deg": 5, "model_kwargs": {}},True, "probit_5"), 
                                        #("rf", {"deg": 1, "model_kwargs": RF_KWARGS_RO},True, "rf_10"), 
                                        #("rf", {"deg": 3, "model_kwargs": RF_KWARGS_RO},True, "rf_30"), 
                                        #("rf", {"deg": 1, "model_kwargs": RF_KWARGS_RO2},True, "rf_12"), 
                                        #("rf", {"deg": 3, "model_kwargs": RF_KWARGS_RO2},True, "rf_32"), 
                                        #("rf", {"deg": 1, "model_kwargs": RF_KWARGS_RO2},False, "rf_12"), 
                                        ("rf", {"deg": 1, "model_kwargs": RF_KWARGS_3},False, "rf_13"), 
                                        #("rf", {"deg": 1, "model_kwargs": RF_KWARGS_4},False, "rf_14"), 
                                        #("rf", {"deg": 1, "model_kwargs": RF_KWARGS_MAX},False, "rf_1m"), 
            
                                        ]:
            try:
                if log_ykappa:
                #ys_kappa = ys.copy()*1/10000
                    ys_kappa = numpy.log10(1+ys.copy())
                else:
                    ys_kappa, _ = boxcox(df_[y_col].clip(1, numpy.inf))
                    ys_kappa = pandas.Series(ys_kappa, df_.index)
                    ys_kappa = ys_kappa/ys_kappa.median()-1
                    ys_kappa.name = y_col
                    #ys_kappa = ys
                    #ys_kappa = ys.copy()*1/10000
               
                ws, cv0, cv1 = get_kappa(xs, ys_kappa, zs, ds, model_name, model_kwargs)
            except Exception as e:
                #raise AssertionError("")
                print("Exception during kappa fit", e)
                continue
            X = df_[x_cols_m+[d_col]]
            y = ys
            w = ws
            w = ws/sum(ws)
            assert all(X.index==y.index)
            assert all(y.index==w.index)


            for q in [.15, .25, .5, .75, .85]:
            #for q in [.15, .25, .5]:
                print(i, d, model_name, model_str, q, cv0, cv1)
                this_result = get_result(QuantileRegressor, {"type": f"QTE(sex={d}){q}({model_str}, log_kappa {log_ykappa})", "kappa_cv0": cv0, "kappa_cv1": cv1},
                                model_kwargs={"quantile": q, "alpha":0, "solver": "highs-ds", "fit_intercept":False},
                                fit_kwargs={"X": X, "y": y, "sample_weight": w},
                                params_attr = "params", bse_attr=None)

                all_results_bootstrap_2slsqte+=this_result
clear_output()

In [27]:
df_results_bootstrap_2slsqte = pandas.DataFrame(all_results_bootstrap_2slsqte)

In [28]:
pandas.set_option('display.max_columns', 500)

df_results_param = df_results_bootstrap_2slsqte.groupby(["type", "param_type"], as_index=False).mean()
df_results_param["param_type"] = "coef"
df_results_dev = df_results_bootstrap_2slsqte.groupby(["type", "param_type"], as_index=False).std()
df_results_dev["param_type"] = "se"
df_results = pandas.concat([df_results_param, df_results_dev], axis=0)
get_final_table(df_results, add_kappa_cv=True)

value                                           \
type                 2SLS(sex=1) QTE(sex=1)0.15(logit_7, log_kappa False)   
variable  param_type                                                        
d         coef          1474.859                                  161.208   
          se             654.282                                  398.240   
hsorged   coef          3798.728                                  596.713   
          se             728.760                                  424.462   
black     coef         -2086.998                                 -232.721   
          se             455.583                                  367.184   
hispanic  coef           642.830                                  423.065   
          se             287.016                                  437.019   
married   coef          6801.422                                 1521.579   
          se             266.747                                  571.236   
wkless13  coef         -6128.656                                -2106.390   
          se             330.741                                  196.052   
constant  coef         10214.103                                  245.258   
          se            1920.542                                 1335.858   
kappa_cv0 coef               NaN                                    0.671   
          se                 NaN                                    0.005   
kappa_cv1 coef               NaN                                    0.056   
          se                 NaN                                    0.007   

                                                             \
type                 QTE(sex=1)0.15(rf_13, log_kappa False)   
variable  param_type                                          
d         coef                                      561.496   
          se                                        323.821   
hsorged   coef                                      684.527   
          se                                        344.148   
black     coef                                      -93.057   
          se                                        369.733   
hispanic  coef                                      245.641   
          se                                        208.909   
married   coef                                     1397.645   
          se                                        491.381   
wkless13  coef                                    -2243.134   
          se                                        305.520   
constant  coef                                       18.928   
          se                                       1462.786   
kappa_cv0 coef                                        0.622   
          se                                          0.005   
kappa_cv1 coef                                        0.045   
          se                                          0.004   

                                                               \
type                 QTE(sex=1)0.25(logit_7, log_kappa False)   
variable  param_type                                            
d         coef                                        513.661   
          se                                          381.391   
hsorged   coef                                       1177.641   
          se                                          586.219   
black     coef                                       -223.595   
          se                                          452.953   
hispanic  coef                                       1515.093   
          se                                          487.501   
married   coef                                       3137.728   
          se                                          671.373   
wkless13  coef                                      -4381.529   
          se                                          449.810   
constant  coef                                       2879.839   
          se                                 

### Conclusions

- There are three main findings in the original paper, all of which we can reproduce with the above method.

  1) IV and median effect are indistinguishable and in the range 1750 (1000).
  2) On the upper quantiles, the effects are smaller compared to pure quantile regression but significantly positive.
  2) However, the effects on the 15% and 25% percentiles are significantly smaller and non-significant.
 